In [ ]:
arrow::read_parquet(here("data", "featureless.parquet"), as_data_frame = TRUE) -> atp
arrow::read_parquet(here("data", "atpTrans.parquet")) -> atpFull
library(skimr)

In [ ]:
# features q faltam:
# tipo do torneio (afonse)
# frequencia do torneio
# premio (precisa eda) 
# distancia da ronda final (chaves)
# categoria de rank (eda)
# ultimo torneio do jogador
# ngames ate o ponto

# Categorias de torneio

Superiores:
    - Grand Slams (melhor de 5)
    - Masters 1000 (considerado o 5º Grand Slam) (melhor de 3)

Intermédios:
    - ATP 500 (terceiro nível de torneios, distribuindo 500 pontos ao campeão)
    - ATP 250 (distribui 250 pontos ao campeão)
    - ATP Finals (reúne os oito primeiros colocados do ranking ao final da temporada. Possui grande importância no calendário dos tenistas, 
                  algo próximo aos torneios Masters 1000)
    - Copa Davis (Onde ocorre a dispuita tradicional entre países)

Menores:
    - Challenger (tenistas tentam somar pontos que os permitam participar de competições mais importantes, ingressando no circuito da ATP)
    - Futures (torneios de categoria inferior ao nível Challenger)

# Número de jogos jogados

In [ ]:
atp %>%
    group_by(winner.name, winner.id) %>% 
    summarise(player.nwins = n()) %>% 
    rename(player.name = winner.name, player.id = winner.id) -> a
atp %>%
    group_by(looser.name, looser.id) %>% 
    summarise(player.nlosses = n()) %>% 
    rename(player.name = looser.name, player.id = looser.id) -> b

full_join(a, b) %>% mutate(
    player.nwins   = ifelse(player.nwins %>% is.na, 0, player.nwins),
    player.nlosses = ifelse(player.nlosses %>% is.na, 0, player.nlosses),
    player.njogos  = player.nwins + player.nlosses,
    player.winrate = player.nwins*100 / player.njogos,
) -> playerRate
playerRate

group_by: 2 grouping variables (winner.name, winner.id)

summarise: now 901 rows and 3 columns, one group variable remaining (winner.name)

rename: renamed 2 variables (player.name, player.id)

group_by: 2 grouping variables (looser.name, looser.id)

summarise: now 1,194 rows and 3 columns, one group variable remaining (looser.name)

rename: renamed 2 variables (player.name, player.id)

Joining with `by = join_by(player.name, player.id)`
full_join: added one column (player.nlosses)

           > rows only in x      17

           > rows only in y     310

           > matched rows       884

           >                 =======

           > rows total       1,211

mutate (grouped): converted 'player.nwins' from integer to double (310 fewer NA)

                  converted 'player.nlosses' from integer to double (17 fewer NA)

                  new variable 'player.njogos' (double) with 79 unique values and 0% NA

                  new variable 'player.winrate' (double) with 166 unique

player.name,player.id,player.nwins,player.nlosses,player.njogos,player.winrate
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Aaron Addison,ad06,8,37,45,17.77778
Aaron Krickstein,k023,20,6,26,76.92308
Adam Feeney,f491,39,54,93,41.93548
Adrian Garcia,g414,3,3,6,50.00000
Adrian Mannarino,me82,4,5,9,44.44444
Adrian Voinea,v166,3,2,5,60.00000
Agustin Calleri,c478,1,2,3,33.33333
Akira Santillan,sq80,16,19,35,45.71429
Albert Chang,c308,1,2,3,33.33333


In [ ]:
atp %>% 
    left_join(playerRate, by=join_by(winner.name == player.name, winner.id == player.id)) %>% 
    rename(
        winner.Anwins   = player.nwins, 
        winner.Anlosses = player.nlosses, 
        winner.Anjogos  = player.njogos, 
        winner.Awinrate = player.winrate
    ) %>% left_join(playerRate, by=join_by(looser.name == player.name, looser.id == player.id)) %>% 
    rename(
        looser.Anwins   = player.nwins, 
        looser.Anlosses = player.nlosses, 
        looser.Anjogos  = player.njogos, 
        looser.Awinrate = player.winrate
    ) %>% mutate(
        match.AnjogosDiff = winner.Anjogos - looser.Anjogos
    ) -> atpP
atpP %>% glimpse()

rename: renamed 4 variables (winner.Anwins, winner.Anlosses, winner.Anjogos, winner.Awinrate)

rename: renamed 4 variables (looser.Anwins, looser.Anlosses, looser.Anjogos, looser.Awinrate)

mutate: new variable 'match.AnjogosDiff' (double) with 334 unique values and 0% NA



Rows: 6,365
Columns: 37
$ winner.name             <chr> "Novak Djokovic", "Roberto Bautista Agut", "Da…
$ winner.id               <chr> "d643", "bd06", "e687", "mm58", "gb88", "l987"…
$ winner.rank             <int> 2, 9, 42, 5, 11, 34, 17, 30, 2, 9, 18, 18, 24,…
$ winner.bornAt           <fct> Serbia, Spain, United Kingdom, Russia, Belgium…
$ match.domHands          <fct> Right-Handed_Left-Handed, Right-Handed_Right-H…
$ match.backHands         <fct> Two-Handed Backhand_Two-Handed Backhand, Two-H…
$ winner.height           <dbl> 188, 183, 175, 198, 180, 183, 198, 193, 188, 1…
$ looser.name             <chr> "Rafael Nadal", "Dusan Lajovic", "Alex de Mina…
$ looser.id               <chr> "n409", "l987", "dh58", "sm37", "n409", "ag37"…
$ looser.rank             <int> 1, 34, 18, 14, 1, 21, 25, 53, 15, 160, 7, 15, …
$ looser.bornAt           <fct> Spain, Serbia, Australia, Argentina, Spain, Ca…
$ looser.height           <dbl> 185, 183, 183, 170, 185, 193, 183, 188, 185, 1…
$ match.tourname

In [ ]:
atpFull %>%
    group_by(winner.name, winner.id) %>% 
    summarise(player.nwins = n()) %>% 
    rename(player.name = winner.name, player.id = winner.id) -> a
atpFull %>%
    group_by(looser.name, looser.id) %>% 
    summarise(player.nlosses = n()) %>% 
    rename(player.name = looser.name, player.id = looser.id) -> b

full_join(a, b) %>% mutate(
    player.nwins   = ifelse(player.nwins %>% is.na, 0, player.nwins),
    player.nlosses = ifelse(player.nlosses %>% is.na, 0, player.nlosses),
    player.njogos  = player.nwins + player.nlosses,
    player.winrate = player.nwins*100 / player.njogos,
) -> playerRateF
playerRateF

group_by: 2 grouping variables (winner.name, winner.id)

summarise: now 12,001 rows and 3 columns, one group variable remaining (winner.name)

rename: renamed 2 variables (player.name, player.id)

group_by: 2 grouping variables (looser.name, looser.id)

summarise: now 22,905 rows and 3 columns, one group variable remaining (looser.name)

rename: renamed 2 variables (player.name, player.id)

Joining with `by = join_by(player.name, player.id)`
full_join: added one column (player.nlosses)

           > rows only in x      475

           > rows only in y   11,379

           > matched rows     11,526

           >                 ========

           > rows total       23,380

mutate (grouped): converted 'player.nwins' from integer to double (11379 fewer NA)

                  converted 'player.nlosses' from integer to double (475 fewer NA)

                  new variable 'player.njogos' (double) with 889 unique values and 0% NA

                  new variable 'player.winrate' (double) wi

player.name,player.id,player.nwins,player.nlosses,player.njogos,player.winrate
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
A.J. Catanzariti,ci19,21,38,59,35.593220
AC Hans Van Swol,NA,1,0,1,100.000000
Aaro Pollanen,ph59,5,19,24,20.833333
Aaron Cohen,NA,1,10,11,9.090909
Aaron Addison,ad06,44,96,140,31.428571
Aaron Banasik,bl87,3,11,14,21.428571
Aaron Bishop,bf00,1,2,3,33.333333
Aaron Cohen,c0hf,2,10,12,16.666667
Aaron Cortes Alcaraz,cg47,43,98,141,30.496454


In [ ]:
atpP %>% 
    left_join(playerRateF, by=join_by(winner.name == player.name, winner.id == player.id)) %>% 
    rename(
        winner.nwins   = player.nwins, 
        winner.nlosses = player.nlosses, 
        winner.njogos  = player.njogos, 
        winner.winrate = player.winrate
    ) %>% left_join(playerRate, by=join_by(looser.name == player.name, looser.id == player.id)) %>% 
    rename(
        looser.nwins   = player.nwins, 
        looser.nlosses = player.nlosses, 
        looser.njogos  = player.njogos, 
        looser.winrate = player.winrate
    ) %>% mutate(
        match.njogosDiff = winner.njogos - looser.njogos
    ) -> atpPP
atpPP %>% glimpse()

rename: renamed 4 variables (winner.nwins, winner.nlosses, winner.njogos, winner.winrate)

rename: renamed 4 variables (looser.nwins, looser.nlosses, looser.njogos, looser.winrate)

mutate: new variable 'match.njogosDiff' (double) with 1,141 unique values and 0% NA



Rows: 6,365
Columns: 46
$ winner.name             <chr> "Novak Djokovic", "Roberto Bautista Agut", "Da…
$ winner.id               <chr> "d643", "bd06", "e687", "mm58", "gb88", "l987"…
$ winner.rank             <int> 2, 9, 42, 5, 11, 34, 17, 30, 2, 9, 18, 18, 24,…
$ winner.bornAt           <fct> Serbia, Spain, United Kingdom, Russia, Belgium…
$ match.domHands          <fct> Right-Handed_Left-Handed, Right-Handed_Right-H…
$ match.backHands         <fct> Two-Handed Backhand_Two-Handed Backhand, Two-H…
$ winner.height           <dbl> 188, 183, 175, 198, 180, 183, 198, 193, 188, 1…
$ looser.name             <chr> "Rafael Nadal", "Dusan Lajovic", "Alex de Mina…
$ looser.id               <chr> "n409", "l987", "dh58", "sm37", "n409", "ag37"…
$ looser.rank             <int> 1, 34, 18, 14, 1, 21, 25, 53, 15, 160, 7, 15, …
$ looser.bornAt           <fct> Spain, Serbia, Australia, Argentina, Spain, Ca…
$ looser.height           <dbl> 185, 183, 183, 170, 185, 193, 183, 188, 185, 1…
$ match.tourname

In [ ]:
atpFull %>% 
    group_by(tournament.location, tournament.name, tournament.date_start) %>%
    summarise() %>%
    group_by(tournament.location) %>%
    summarise(tournament.locationFreq = n()) -> tfreq

atpPP %>% 
    left_join(tfreq, by=join_by(winner.bornAt == tournament.location)) %>% 
    rename(winner.bornAT.tournFreq = tournament.locationFreq) %>%
    left_join(tfreq, by=join_by(looser.bornAt == tournament.location)) %>% 
    rename(looser.bornAT.tournFreq = tournament.locationFreq) %>%
    mutate(
        winner.bornAt.hasTourn = winner.bornAT.tournFreq != 0,
        looser.bornAt.hasTourn = looser.bornAT.tournFreq != 0,
        match.playersBorn.hasTournXor = winner.bornAt.hasTourn == looser.bornAt.hasTourn,
        # match.playersBorn.hasTournAny = any(winner.bornAt.hasTourn, looser.bornAt.hasTourn), #all T
        match.playersBorn.tournFreqDiff = winner.bornAt.hasTourn - looser.bornAt.hasTourn,
    ) -> atpPPT
atpPPT %>% glimpse()

group_by: 3 grouping variables (tournament.location, tournament.name, tournament.date_start)

summarise: now 22,234 rows and 3 columns, 2 group variables remaining (tournament.location, tournament.name)

group_by: one grouping variable (tournament.location)

summarise: now 139 rows and 2 columns, ungrouped

rename: renamed one variable (winner.bornAT.tournFreq)

rename: renamed one variable (looser.bornAT.tournFreq)

mutate: new variable 'winner.bornAt.hasTourn' (logical) with 2 unique values and <1% NA

        new variable 'looser.bornAt.hasTourn' (logical) with 2 unique values and <1% NA

        new variable 'match.playersBorn.hasTournXor' (logical) with 2 unique values and <1% NA

        new variable 'match.playersBorn.tournFreqDiff' (integer) with 2 unique values and <1% NA



Rows: 6,365
Columns: 52
$ winner.name                     <chr> "Novak Djokovic", "Roberto Bautista Ag…
$ winner.id                       <chr> "d643", "bd06", "e687", "mm58", "gb88"…
$ winner.rank                     <int> 2, 9, 42, 5, 11, 34, 17, 30, 2, 9, 18,…
$ winner.bornAt                   <chr> "Serbia", "Spain", "United Kingdom", "…
$ match.domHands                  <fct> Right-Handed_Left-Handed, Right-Handed…
$ match.backHands                 <fct> Two-Handed Backhand_Two-Handed Backhan…
$ winner.height                   <dbl> 188, 183, 175, 198, 180, 183, 198, 193…
$ looser.name                     <chr> "Rafael Nadal", "Dusan Lajovic", "Alex…
$ looser.id                       <chr> "n409", "l987", "dh58", "sm37", "n409"…
$ looser.rank                     <int> 1, 34, 18, 14, 1, 21, 25, 53, 15, 160,…
$ looser.bornAt                   <chr> "Spain", "Serbia", "Australia", "Argen…
$ looser.height                   <dbl> 185, 183, 183, 170, 185, 193, 183, 188…
$ match.tourname

In [ ]:
# ultimo torneio do jogador
atpFull %>% 
    group_by(winner.name, winner.id, tournament.name, tournament.date_start) %>% 
    summarise() %>%
    arrange(tournament.date_start) %>% 
    group_by(winner.name, winner.id) %>%
    mutate(
        winner.nTournament         = row_number(),
        winner.lastTournamentDate  = lag(tournament.date_start),
        winner.lastTournamentDate  = ifelse(winner.lastTournamentDate %>% is.na, tournament.date_start, winner.lastTournamentDate),
        winner.lastTournamentDelay = -(tournament.date_start %--% winner.lastTournamentDate) %>% as.duration()
    ) %>% arrange(winner.name, winner.id) -> n
atpPPT %>% left_join(n) -> atpPPTL
# atpFull %>% 
#     group_by(looser.name, looser.id, tournament.name, tournament.date_start) %>% 
#     summarise() %>%
#     arrange(tournament.date_start) %>% 
#     group_by(looser.name, looser.id) %>%
#      mutate(
#         looser.nTournament         = row_number(),
#         looser.lastTournamentDate  = lag(tournament.date_start),
#         looser.lastTournamentDate  = ifelse(looser.lastTournamentDate %>% is.na, tournament.date_start, looser.lastTournamentDate),
#         looser.lastTournamentDelay = (tournament.date_start - winner.lastTournamentDate) %>% make_difftime(units = "weeks")
#     ) %>% arrange(looser.name, looser.id) -> n
# atpPPTLL <- atpPPTL %>% left_join(n)
atpPPTL %>% glimpse()

group_by: 4 grouping variables (winner.name, winner.id, tournament.name, tournament.date_start)

summarise: now 356,796 rows and 4 columns, 3 group variables remaining (winner.name, winner.id, tournament.name)

group_by: 2 grouping variables (winner.name, winner.id)

mutate (grouped): new variable 'winner.nTournament' (integer) with 361 unique values and 0% NA

                  new variable 'winner.lastTournamentDate' (double) with 2,709 unique values and 0% NA

                  new variable 'winner.lastTournamentDelay' (double) with 52,512 unique values and 0% NA

Joining with `by = join_by(winner.name, winner.id, tournament.name, tournament.date_start)`


Rows: 6,365
Columns: 55
$ winner.name                     <chr> "Novak Djokovic", "Roberto Bautista Ag…
$ winner.id                       <chr> "d643", "bd06", "e687", "mm58", "gb88"…
$ winner.rank                     <int> 2, 9, 42, 5, 11, 34, 17, 30, 2, 9, 18,…
$ winner.bornAt                   <chr> "Serbia", "Spain", "United Kingdom", "…
$ match.domHands                  <fct> Right-Handed_Left-Handed, Right-Handed…
$ match.backHands                 <fct> Two-Handed Backhand_Two-Handed Backhan…
$ winner.height                   <dbl> 188, 183, 175, 198, 180, 183, 198, 193…
$ looser.name                     <chr> "Rafael Nadal", "Dusan Lajovic", "Alex…
$ looser.id                       <chr> "n409", "l987", "dh58", "sm37", "n409"…
$ looser.rank                     <int> 1, 34, 18, 14, 1, 21, 25, 53, 15, 160,…
$ looser.bornAt                   <chr> "Spain", "Serbia", "Australia", "Argen…
$ looser.height                   <dbl> 185, 183, 183, 170, 185, 193, 183, 188…
$ match.tourname

In [ ]:
atpFull %>% 
    group_by(winner.name, winner.id, tournament.name, tournament.date_start) %>% 
    summarise() %>%
    arrange(tournament.date_start) %>% 
    rename(player.name = winner.name, player.id = winner.id) -> a
atpFull %>% 
    group_by(looser.name, looser.id, tournament.name, tournament.date_start) %>% 
    summarise() %>%
    arrange(tournament.date_start) %>% 
    rename(player.name = looser.name, player.id = looser.id) -> b
bind_rows(a, b) %>% 
   group_by(player.name, player.id) %>%
   mutate(
       player.nTournament          = row_number(),
       player.lastTournamentDate   = ifelse(player.nTournament == 1, tournament.date_start, lag(tournament.date_start)) %>% as.Date,
       player.lastTournamentDelayW = -interval(tournament.date_start, player.lastTournamentDate) / dweeks(1)#as weeks
   ) %>% select(-player.lastTournamentDate) -> ab

atpPPT %>% 
    left_join(ab, by=join_by(
        winner.name == player.name, 
        winner.id == player.id, 
        tournament.date_start == tournament.date_start,
        tournament.name == tournament.name
    ), relationship = "many-to-many") %>%
    rename_with(\(x) str_replace(x, "player", "winner"), starts_with("player")) %>%
    left_join(ab, by=join_by(
        looser.name == player.name, 
        looser.id == player.id, 
        tournament.date_start == tournament.date_start,
        tournament.name == tournament.name
    ), relationship = "many-to-many") %>%
    rename_with(\(x) str_replace(x, "player", "looser"), starts_with("player")) -> atpPPTD
atpPPTD %>% glimpse()

group_by: 4 grouping variables (winner.name, winner.id, tournament.name, tournament.date_start)

summarise: now 356,796 rows and 4 columns, 3 group variables remaining (winner.name, winner.id, tournament.name)

rename: renamed 2 variables (player.name, player.id)

group_by: 4 grouping variables (looser.name, looser.id, tournament.name, tournament.date_start)

summarise: now 677,391 rows and 4 columns, 3 group variables remaining (looser.name, looser.id, tournament.name)

rename: renamed 2 variables (player.name, player.id)

group_by: 2 grouping variables (player.name, player.id)

mutate (grouped): new variable 'player.nTournament' (integer) with 883 unique values and 0% NA

                  new variable 'player.lastTournamentDate' (Date) with 2,714 unique values and 0% NA

                  new variable 'player.lastTournamentDelayW' (double) with 2,013 unique values and 0% NA

select: dropped one variable (player.lastTournamentDate)

rename_with: renamed 2 variables (winner.nTournamen

Rows: 17,926
Columns: 56
$ winner.name                     <chr> "Novak Djokovic", "Novak Djokovic", "R…
$ winner.id                       <chr> "d643", "d643", "bd06", "bd06", "e687"…
$ winner.rank                     <int> 2, 2, 9, 9, 42, 42, 42, 42, 5, 5, 5, 5…
$ winner.bornAt                   <chr> "Serbia", "Serbia", "Spain", "Spain", …
$ match.domHands                  <fct> Right-Handed_Left-Handed, Right-Handed…
$ match.backHands                 <fct> Two-Handed Backhand_Two-Handed Backhan…
$ winner.height                   <dbl> 188, 188, 183, 183, 175, 175, 175, 175…
$ looser.name                     <chr> "Rafael Nadal", "Rafael Nadal", "Dusan…
$ looser.id                       <chr> "n409", "n409", "l987", "l987", "dh58"…
$ looser.rank                     <int> 1, 1, 34, 34, 18, 18, 18, 18, 14, 14, …
$ looser.bornAt                   <chr> "Spain", "Spain", "Serbia", "Serbia", …
$ looser.height                   <dbl> 185, 185, 183, 183, 183, 183, 183, 183…
$ match.tournam

In [ ]:
# atpPPTD %>% SmartEDA::ExpReport(Target = "match.setsCount", op_file="temp.html")
# atpPPTD %>% SmartEDA::ExpNumViz(target = "match.setsCount", col = c("Red", "Blue"), fname="temp.scatters", scatter=T)

Coisas a notar: 

* tournament duration é melhor ser categórico ou ordinal
* height sugere ser melhor ordinal ou range
* prize precisa de mais eda mas probs deve ser range ou até binario
* freq tourneio deve ser preciso ser categorico ou normalizar 
* diff rank deve ser normalizado tmb (posivitive skew)
* possivelmente seja bom dar oversample de domHands e tournament.ground (embora eles estejam igualmene distribuidos na var (n me lembro se é bom ou mau)) 
* match.setCount está 2/3 3 jogos
* secalhar seria bom agrupar backhands, tournament.quarter
* as categoricas estão todas igualmente distribuidas em termos da target

In [ ]:
atpPPTD %>% 
    group_by(winner.height) %>%
    summarise(n = n(), prop = n*100/nrow(.)) %>%
    mutate(propSum = cumsum(prop), range = propSum %/% 20)

group_by: one grouping variable (winner.height)

summarise: now 18 rows and 3 columns, ungrouped

mutate: new variable 'propSum' (double) with 18 unique values and 0% NA

        new variable 'range' (double) with 6 unique values and 0% NA



winner.height,n,prop,propSum,range
<dbl>,<int>,<dbl>,<dbl>,<dbl>
163,11,0.06136338,0.06136338,0
168,186,1.03759902,1.09896240,0
170,238,1.32768046,2.42664287,0
173,198,1.10454089,3.53118376,0
175,773,4.31217226,7.84335602,0
178,1303,7.26877162,15.11212764,0
180,2194,12.23920562,27.35133326,1
183,3830,21.36561419,48.71694745,2
185,2118,11.81524043,60.53218788,3


In [ ]:
atpPPTD %>% group_by(atpPPTD$match.backHands) %>% summarise(n = n(),  prop = n*100/nrow(.))

group_by: one grouping variable (atpPPTD$match.backHands)

summarise: now 9 rows and 3 columns, ungrouped



atpPPTD$match.backHands,n,prop
<fct>,<int>,<dbl>
One-Handed Backhand_One-Handed Backhand,271,1.511771
One-Handed Backhand_Two-Handed Backhand,1272,7.095838
One-Handed Backhand_Unknown Backhand,693,3.865893
Two-Handed Backhand_One-Handed Backhand,1216,6.783443
Two-Handed Backhand_Two-Handed Backhand,7677,42.826063
Two-Handed Backhand_Unknown Backhand,1532,8.546246
Unknown Backhand_One-Handed Backhand,607,3.386143
Unknown Backhand_Two-Handed Backhand,1383,7.715051
Unknown Backhand_Unknown Backhand,3275,18.269553


In [ ]:
atpPPTD %>% group_by(atpPPTD$tournament.quarter) %>% summarise(n = n(),  prop = n*100/nrow(.))

group_by: one grouping variable (atpPPTD$tournament.quarter)

summarise: now 4 rows and 3 columns, ungrouped



atpPPTD$tournament.quarter,n,prop
<fct>,<int>,<dbl>
1,9655,53.860315
2,186,1.037599
3,1463,8.161330
4,6622,36.940756


In [ ]:
atpPPTD %>% arrange(winner.height) %>% mutate(
    tournament.durationF  = tournament.duration %>% as_factor(),
    tournament.durationFL = tournament.durationF %>% fct_lump_n(1),
    winner.heightF        = winner.height %>% as_factor() %>% fct_inseq(),
    looser.heightF        = looser.height %>% as_factor() %>% fct_inseq(),
    winner.heightFC       = case_when( # division on 33/66
        winner.height <= 180 ~ "]163, 180]",
        winner.height <= 186 ~ "]180, 186]",
        TRUE                 ~ "]186, 211]"
    ) %>% as_factor() %>% fct_inorder()
) %>% arrange(looser.height) %>% mutate(
    looser.heightFC       = case_when(
        looser.height <= 180 ~ "]163, 180]",
        looser.height <= 186 ~ "]180, 186]",
        TRUE                 ~ "]186, 211]"
    ) %>% as_factor() %>% fct_inorder(),
    match.heightFCvs = paste(winner.heightFC, looser.heightFC, sep=" vs ") %>% as_factor(),
) %>% mutate(
    match.backHandsL1 = match.backHands %>% fct_lump_n(1),
    match.backHandsL2 = match.backHands %>% fct_lump_n(2),
    tournament.quarterL1 = tournament.quarter %>% fct_lump_n(1),
    tournament.quarterL2 = tournament.quarter %>% fct_lump_n(2)
)-> featurefull



#featurefull %>% skim(
#    tournament.durationF, 
#    tournament.durationFL, 
#    winner.heightF, 
#    looser.heightF,
#    winner.heightFC,
#    looser.heightFC,
#    match.heightFCvs,
#    match.backHandsL1,
#    match.backHandsL2,
#    tournament.quarterL1,
#    tournament.quarterL2,
#)

mutate: new variable 'tournament.durationF' (factor) with 5 unique values and 0% NA

        new variable 'tournament.durationFL' (factor) with 2 unique values and 0% NA

        new variable 'winner.heightF' (factor) with 18 unique values and 0% NA

        new variable 'looser.heightF' (factor) with 18 unique values and 0% NA

        new variable 'winner.heightFC' (factor) with 3 unique values and 0% NA

mutate: new variable 'looser.heightFC' (factor) with 3 unique values and 0% NA

        new variable 'match.heightFCvs' (factor) with 9 unique values and 0% NA

mutate: new variable 'match.backHandsL1' (factor) with 2 unique values and 0% NA

        new variable 'match.backHandsL2' (factor) with 3 unique values and 0% NA

        new variable 'tournament.quarterL1' (factor) with 2 unique values and 0% NA

        new variable 'tournament.quarterL2' (factor) with 3 unique values and 0% NA



In [ ]:
featurefull %>% arrow::write_parquet(here("data", "featurefull.parquet"))